## Trabajo Final: Procesamiento datos con Python

### 1. Objetivos:
    - Aprender a usar try except para evitar que las excepciones detengan nuestros programas
 
---
    
### 2. Desarrollo:

In [1]:
#Intalación de componentes y librerias adicionales
#Libreria adicional para manejo de archivos parquet
!pip install pyarrow 
!pip install fastparquet
!pip install openpyxl
!pip install findspark
!pip install fugue[sql]
!pip install -U pandasql

In [73]:
#Se cargan las librerias utilizadas para los procesamientos
from pandasql import sqldf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fugue_sql import fsql

In [3]:
#Definición de variables para las diferentes rutas de almacenamiento
vl_ruta_zona_bronze = "Dataset/almacenamiento/bronze/"
vl_ruta_zona_silver = "Dataset/almacenamiento/silver/"
vl_ruta_zona_gold = "Dataset/almacenamiento/gold/"

In [4]:
#Lectura datos
def fu_leerdatoszonabronze(pi_nombrearchivo, pi_tipoarchivo, separador=None, codificacion=None, nombrehojaexcel=None):
    #pi_tipoarchivo = 1 -> Plano, separador tabulador y con encoding personalizado
    #pi_tipoarchivo = 2 -> Archivo parquet
    #pi_tipoarchivo = 3 -> Archivo json
    #pi_tipoarchivo = 4 -> csv, separador personalizado
    #pi_tipoarchivo = 5 -> excel
    
    vl_dat_fechaini = datetime.now()
    vl_df_insumo = None
        
    try:
        if pi_tipoarchivo==1:
            vl_df_insumo = pd.read_csv(f"{vl_ruta_zona_bronze}{pi_nombrearchivo}", sep=separador, encoding=codificacion) 
            
        if pi_tipoarchivo==2:
            vl_df_insumo = pd.read_parquet(f"{vl_ruta_zona_bronze}{pi_nombrearchivo}")
            
        if pi_tipoarchivo==3:
            vl_df_insumo = pd.read_json(f"{vl_ruta_zona_bronze}{pi_nombrearchivo}")
            
        if pi_tipoarchivo==4:
            vl_df_insumo = pd.read_csv(f"{vl_ruta_zona_bronze}{pi_nombrearchivo}", sep=separador)
            
        if pi_tipoarchivo==5:
            vl_df_insumo = pd.read_excel(f"{vl_ruta_zona_bronze}{pi_nombrearchivo}", sheet_name=nombrehojaexcel)
        
        #vl_df_insumo_principal = pd.read_csv(f"{vl_ruta_zona_bronze}insumo_principal_total.zip", sep ='\t',encoding='ISO-8859-1') # 0:06:09.582346 seg -> Filas/columnas: (28'404.277, 42)
        vl_str_mensaje = 'Carga archivo "' + pi_nombrearchivo + '" (FI, FF, T): ' + str(vl_dat_fechaini) + ', ' \
                         + str(datetime.now()) + ', ' + str(datetime.now() - vl_dat_fechaini) + '\n' \
                         + 'Filas/columnas: ' + str(vl_df_insumo.shape) + '\n'
        
    except Exception as e:
        vl_str_mensaje = str(e)
        
    finally:
        return vl_df_insumo, vl_str_mensaje
        #Marca como eliminadas en memoria, las variables utilizadas
        vl_str_mensaje = None
        vl_df_insumo = None
        
        
def fu_escribirarchivozonasilver(pi_df, pi_nombrearchivo, pi_tipoarchivo, separador=None, codificacion=None):
    #pi_tipoarchivo = 1 -> Plano, separador personalizado
    #pi_tipoarchivo = 2 -> Archivo parquet
    #pi_tipoarchivo = 3 -> Archivo json
    
    vl_dat_fechaini = datetime.now()
    vl_df_insumo = None
        
    try:
        if pi_tipoarchivo==1:
            pi_df.to_csv(f"{vl_ruta_zona_silver}{pi_nombrearchivo}", sep=separador, encoding=codificacion) 
            
        if pi_tipoarchivo==2:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}.parquet"
            pi_df.to_parquet(vl_str_mensaje)
            
        if pi_tipoarchivo==3:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}.json"
            pi_df.to_json(vl_str_mensaje)
        
        #vl_df_insumo_principal = pd.read_csv(f"{vl_ruta_zona_bronze}insumo_principal_total.zip", sep ='\t',encoding='ISO-8859-1') # 0:06:09.582346 seg -> Filas/columnas: (28'404.277, 42)
        vl_str_mensaje = 'Archivo "' + vl_str_mensaje + '" se escribio satisfactoriamente: \n(FI, FF, T) ' \
                         + str(vl_dat_fechaini) + ', ' \
                         + str(datetime.now()) + ', ' + str(datetime.now() - vl_dat_fechaini) + '\n' \
                         + 'Filas/columnas: ' + str(pi_df.shape) + '\n'
        
    except Exception as e:
        vl_str_mensaje = str(e)
        
    finally:
        return vl_str_mensaje
        #Marca como eliminadas en memoria, las variables utilizadas
        vl_str_mensaje = None
        
        
def fu_escribirarchivozonagold(pi_df, pi_nombrearchivo, pi_tipoarchivo, separador=None, codificacion=None, nombrehojaexcel=None):
    #pi_tipoarchivo = 1 -> Plano, separador personalizado
    #pi_tipoarchivo = 2 -> Archivo parquet
    #pi_tipoarchivo = 3 -> Archivo json
    
    vl_dat_fechaini = datetime.now()
    vl_df_insumo = None
        
    try:
        if pi_tipoarchivo==1:
            pi_df.to_csv(f"{vl_ruta_zona_gold}{pi_nombrearchivo}", sep=separador, encoding=codificacion) 
            
        if pi_tipoarchivo==2:
            vl_str_mensaje = f"{vl_ruta_zona_gold}{pi_nombrearchivo}.parquet"
            pi_df.to_parquet(vl_str_mensaje)
            
        if pi_tipoarchivo==3:
            vl_str_mensaje = f"{vl_ruta_zona_gold}{pi_nombrearchivo}.json"
            pi_df.to_json(vl_str_mensaje)
            
        if pi_tipoarchivo==4:
            vl_str_mensaje = f"{vl_ruta_zona_gold}{pi_nombrearchivo}.xlsx"
            pi_df.to_excel(vl_str_mensaje, sheet_name=nombrehojaexcel)
        
        #vl_df_insumo_principal = pd.read_csv(f"{vl_ruta_zona_bronze}insumo_principal_total.zip", sep ='\t',encoding='ISO-8859-1') # 0:06:09.582346 seg -> Filas/columnas: (28'404.277, 42)
        vl_str_mensaje = 'Archivo "' + vl_str_mensaje + '" se escribio satisfactoriamente: \n(FI, FF, T) ' \
                         + str(vl_dat_fechaini) + ', ' \
                         + str(datetime.now()) + ', ' + str(datetime.now() - vl_dat_fechaini) + '\n' \
                         + 'Filas/columnas: ' + str(pi_df.shape) + '\n'
        
    except Exception as e:
        vl_str_mensaje = str(e)
        
    finally:
        return vl_str_mensaje
        #Marca como eliminadas en memoria, las variables utilizadas
        vl_str_mensaje = None
        
        
def fu_leerarchivozonasilver(pi_nombrearchivo, pi_tipoarchivo, separador=None, codificacion=None, nombrehojaexcel=None):
    #pi_tipoarchivo = 1 -> Plano, separador tabulador y con encoding personalizado
    #pi_tipoarchivo = 2 -> Archivo parquet
    #pi_tipoarchivo = 3 -> Archivo json
    #pi_tipoarchivo = 4 -> csv, separador personalizado
    #pi_tipoarchivo = 5 -> excel
    
    vl_dat_fechaini = datetime.now()
    vl_df_insumo = None
        
    try:
        if pi_tipoarchivo==1:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}"
            vl_df_insumo = pd.read_csv(vl_str_mensaje, sep=separador, encoding=codificacion) 
            
        if pi_tipoarchivo==2:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}.parquet"
            vl_df_insumo = pd.read_parquet(vl_str_mensaje)
            
        if pi_tipoarchivo==3:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}.json"
            vl_df_insumo = pd.read_json(vl_str_mensaje)
            
        if pi_tipoarchivo==4:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}.csv"
            vl_df_insumo = pd.read_csv(vl_str_mensaje, sep=separador)
            
        if pi_tipoarchivo==5:
            vl_str_mensaje = f"{vl_ruta_zona_silver}{pi_nombrearchivo}.xlsx"
            vl_df_insumo = pd.read_excel(vl_str_mensaje, sheet_name=nombrehojaexcel)
        
        #vl_df_insumo_principal = pd.read_csv(f"{vl_ruta_zona_bronze}insumo_principal_total.zip", sep ='\t',encoding='ISO-8859-1') # 0:06:09.582346 seg -> Filas/columnas: (28'404.277, 42)
        vl_str_mensaje = 'Carga archivo "' + vl_str_mensaje + '" (FI, FF, T): \n' + str(vl_dat_fechaini) + ', ' \
                         + str(datetime.now()) + ', ' + str(datetime.now() - vl_dat_fechaini) + '\n' \
                         + 'Filas/columnas: ' + str(vl_df_insumo.shape) + '\n'
        
    except Exception as e:
        vl_str_mensaje = str(e)
        
    finally:
        return vl_df_insumo, vl_str_mensaje
        #Marca como eliminadas en memoria, las variables utilizadas
        vl_str_mensaje = None
        vl_df_insumo = None
    
   

In [131]:
#Carga de los insumos y maestros de la zona bronze, para iniciar los procesamientos
vl_df_insumo_principal, vl_str_mensaje = fu_leerdatoszonabronze("insumo_principal.zip", 1, "\t", "ISO-8859-1") #Carga archivo insumo prueba basica, # 0:00:00.006983 seg
#vl_df_insumo_principal, vl_str_mensaje = fu_leerdatoszonabronze("insumo_principal_total.zip", 1, "\t", "ISO-8859-1") #Carga archivo insumo prueba basica, # 0:00:00.006983 seg
print(vl_str_mensaje)

vl_df_maestro_pdv, vl_str_mensaje = fu_leerdatoszonabronze("PuntosDeVenta.parquet", 2) #Carga archivo maestro de PDV
print(vl_str_mensaje)

vl_df_maestro_agrupamientoporprocesamiento, vl_str_mensaje = fu_leerdatoszonabronze("AgrupacionXProcesamiento.json", 3) #Carga archivo maestro de agrupamientos
print(vl_str_mensaje)
#vl_df_maestro_agrupamientoporprocesamiento, vl_str_mensaje = fu_leerdatoszonabronze("maestros.xlsx", 5, nombrehojaexcel="AgrupacionXProcesamiento") #Carga archivo maestro de agrupamientos de información
#print(vl_str_mensaje)

vl_df_maestro_ubicaciones, vl_str_mensaje = fu_leerdatoszonabronze("UbicacionesXProcesamiento.csv", 4, ";") #Carga archivo maestro de ubicaciones
print(vl_str_mensaje)

vl_df_maestro_procesamientoxpais, vl_str_mensaje = fu_leerdatoszonabronze("maestros.xlsx", 5, nombrehojaexcel="ProcesamientosXPais") #Carga archivo maestro de procesamiento por pais
print(vl_str_mensaje)

vl_df_maestro_preguntasporprocesamiento, vl_str_mensaje = fu_leerdatoszonabronze("maestros.xlsx", 5, nombrehojaexcel="PreguntasXProcesamiento") #Carga archivo maestro preguntas
print(vl_str_mensaje)


Carga archivo "insumo_principal.zip" (FI, FF, T): 2022-02-10 21:49:49.271610, 2022-02-10 21:49:49.293001, 0:00:00.021391
Filas/columnas: (100, 42)

Carga archivo "PuntosDeVenta.parquet" (FI, FF, T): 2022-02-10 21:49:49.293223, 2022-02-10 21:49:49.303753, 0:00:00.010530
Filas/columnas: (839, 8)

Carga archivo "AgrupacionXProcesamiento.json" (FI, FF, T): 2022-02-10 21:49:49.304756, 2022-02-10 21:49:49.327971, 0:00:00.023215
Filas/columnas: (1804, 9)

Carga archivo "UbicacionesXProcesamiento.csv" (FI, FF, T): 2022-02-10 21:49:49.332560, 2022-02-10 21:49:49.338214, 0:00:00.005654
Filas/columnas: (395, 8)

Carga archivo "maestros.xlsx" (FI, FF, T): 2022-02-10 21:49:49.338214, 2022-02-10 21:49:49.422220, 0:00:00.084006
Filas/columnas: (5, 3)

Carga archivo "maestros.xlsx" (FI, FF, T): 2022-02-10 21:49:49.422220, 2022-02-10 21:49:50.330572, 0:00:00.908352
Filas/columnas: (3898, 11)



In [138]:
#Para visualizar el insumo y maestros cargados
vl_df_insumo_principal.head(3) #Insumo principal
#vl_df_maestro_pdv.head(3000) #Insumo PDV
#vl_df_maestro_ubicaciones.head(3) #Maestro ubicaciones
#vl_df_maestro_procesamientoxpais.head(3) #Maestro procesamientos por pais
#vl_df_maestro_preguntasporprocesamiento.head(3) #Maestro preguntas por procesamiento
#vl_df_maestro_agrupamientoporprocesamiento.head(3000) #Maestro agrupamiento por preocesamiento

#vl_df_insumo_principal.dtypes #Estructura columnas Dataframe
#vl_df_insumo_principal.shape #Cantidad de registros y columnas

,OlaID_Padre,Ola_Padre,SbjNum,FechaEvaluacion,PaisID,Pais,DepartamentoId,Departamento,CiudadID,Ciudad,...,PreguntaIDNivel6,PreguntaDescNivel6,PreguntaIDNivel10,PreguntaDescNivel10,RespuestaID,FechaOla,Flag,Duracion,Respuesta_cruda,Anio_procesamiento
0,202002,Ola Febrero 2020,125577922,2020-02-11 14:39:43,2,Costa Rica,0,Sin Identificar,0,Sin Identificar,...,0,,0,DESCUENTO,10,2020-02-01,3,21,Respuesta cruda: '10',2020
1,202002,Ola Febrero 2020,127226766,2020-02-26 18:14:04,2,Costa Rica,0,Sin Identificar,0,Sin Identificar,...,0,,0,SUPERMERCADO,9,2020-02-01,3,72,Respuesta cruda: '9',2020
2,202002,Ola Febrero 2020,125513354,2020-02-10 21:20:30,2,Costa Rica,0,Sin Identificar,0,Sin Identificar,...,0,,0,BODEGA,cierre,2020-02-01,3,21,Respuesta cruda: '0.916667',2020


In [133]:
#Transformación de maestros e insumos

#MAESTRO DE PDV´s
#Renombra las columnas del maestro de PDV, a partir de los datos de un diccionario
vl_dic_nuevas_columnas = {"Pais":"pais", "Distrito":"distrito", "UsuarioEncargado":"usuario_encargado",
                          "Formato":"formato", "PDVID":"id_pdv", "PDV":"pdv",
                          "HorarioApertura":"horario_apertura", "HorarioCierre":"horario_cierre"}
vl_df_maestro_pdv = vl_df_maestro_pdv.rename(columns=vl_dic_nuevas_columnas)
#Cast sobre algunas columnas
vl_df_maestro_pdv["horario_apertura"] = pd.to_datetime(vl_df_maestro_pdv["horario_apertura"], unit="ms")
vl_df_maestro_pdv["horario_cierre"] = pd.to_datetime(vl_df_maestro_pdv["horario_cierre"], unit="ms")
vl_df_maestro_pdv["id_pdv"] = vl_df_maestro_pdv["id_pdv"].astype(int, errors='ignore')
print('\nMAESTRO DE PDV´s\n' + str(vl_df_maestro_pdv.dtypes))

#MAESTRO DE UBICACIONES
#Renombra las columnas del maestro de PDV, a partir de los datos de un diccionario
vl_dic_nuevas_columnas = {"OlaID":"id_ola", "ID":"id", "ParentID":"id_parent",
                          "Name":"nombre", "Nivel":"nivel", "Variable":"variable",
                          "VariableCod":"codigo_variable", "DescNivel":"descripcion_nivel"}
vl_df_maestro_ubicaciones = vl_df_maestro_ubicaciones.rename(columns=vl_dic_nuevas_columnas)
vl_df_maestro_ubicaciones["id_parent"] = (vl_df_maestro_ubicaciones["id_parent"]*1).astype(float)
print(f"\nMAESTRO DE UBICACIONES\n{vl_df_maestro_ubicaciones.dtypes}")

#MAESTRO PAISES
vl_dic_nuevas_columnas = {"ID":"id", "Pais":"pais", "Procesamiento":"procesamiento"}
vl_df_maestro_procesamientoxpais = vl_df_maestro_procesamientoxpais.rename(columns=vl_dic_nuevas_columnas)
vl_df_maestro_procesamientoxpais["pais"] = vl_df_maestro_procesamientoxpais["pais"].str.replace('Desconocido','Guatemala')
print(f"\nMAESTRO PAISES\n{vl_df_maestro_procesamientoxpais.dtypes}")

#MAESTRO PREGUNTAS POR PROCESAMIENTO
vl_dic_nuevas_columnas = {"OlaID":"id_ola", "GrupoPregunta":"grupo_pregunta", "Descripcion":"descripcion",
                          "Nivel1":"nivel_1", "Nivel2":"nivel_2", "Nivel3":"nivel_3",
                          "Nivel4":"nivel_4", "Nivel5":"nivel_5", "Orden":"orden",
                          "Activo":"activo", "AliasGrupoPregunta":"alias_grupo_pregunta"}
vl_df_maestro_preguntasporprocesamiento = vl_df_maestro_preguntasporprocesamiento.rename(columns=vl_dic_nuevas_columnas)
vl_df_maestro_preguntasporprocesamiento = vl_df_maestro_preguntasporprocesamiento.fillna('')
#vl_df_maestro_preguntasporprocesamiento = vl_df_maestro_preguntasporprocesamiento.dropna()
vl_df_maestro_preguntasporprocesamiento["alias_grupo_pregunta"] = vl_df_maestro_preguntasporprocesamiento["alias_grupo_pregunta"].str.upper().str.strip() #Quita espacio al comienzo y al final
print(f"\nMAESTRO PREGUNTAS POR PROCESAMIENTO\n{vl_df_maestro_preguntasporprocesamiento.dtypes}")

#MAESTRO AGRUPAMIENTO POR PROCESAMIENTO
vl_dic_nuevas_columnas = {"OlaID":"id_ola", "ID":"id", "Name":"descripcion",
                          "Variable":"variable", "VariableCod":"codigo_variable", "ParentID":"id_padre",
                          "Nivel":"nivel", "Orden":"orden", "DescNivel":"descripcion_nivel"}
vl_df_maestro_agrupamientoporprocesamiento = vl_df_maestro_agrupamientoporprocesamiento.rename(columns=vl_dic_nuevas_columnas)
vl_df_maestro_agrupamientoporprocesamiento = vl_df_maestro_agrupamientoporprocesamiento.fillna('')
vl_df_maestro_agrupamientoporprocesamiento["variable"] = vl_df_maestro_agrupamientoporprocesamiento["variable"].str.replace('None','')
vl_df_maestro_agrupamientoporprocesamiento["codigo_variable"] = vl_df_maestro_agrupamientoporprocesamiento["codigo_variable"].str.replace('None','')
print(f"\nMAESTRO PREGUNTAS POR PROCESAMIENTO\n{vl_df_maestro_preguntasporprocesamiento.dtypes}")



MAESTRO DE PDV´s
pais                         object
distrito                     object
usuario_encargado            object
formato                      object
id_pdv                        int32
pdv                          object
horario_apertura     datetime64[ns]
horario_cierre       datetime64[ns]
dtype: object

MAESTRO DE UBICACIONES
id_ola                 int64
id                     int64
id_parent            float64
nombre                object
nivel                  int64
variable              object
codigo_variable       object
descripcion_nivel     object
dtype: object

MAESTRO PAISES
id                int64
pais             object
procesamiento    object
dtype: object

MAESTRO PREGUNTAS POR PROCESAMIENTO
id_ola                   int64
grupo_pregunta          object
descripcion             object
nivel_1                 object
nivel_2                  int64
nivel_3                 object
nivel_4                 object
nivel_5                 object
orden                  

In [168]:
def fu_respuesta_cruda(valor):
    return f"Respuesta cruda: '{valor}'"

def anio_de_procesamiento(valor):
    fecha = pd.to_datetime(valor, unit="ns")    
    return fecha.year

#INSUMO DATA CRUDA Y CRUCE CONTRA DATOS MAESTROS
#Crea columnas y transforma datos
vl_df_insumo_principal = vl_df_insumo_principal.fillna('')
vl_df_insumo_principal['Respuesta_cruda'] = vl_df_insumo_principal['Respuesta'].map(fu_respuesta_cruda)
vl_df_insumo_principal['Anio_procesamiento'] = vl_df_insumo_principal['FechaEvaluacion'].apply(anio_de_procesamiento)

#Aplica filtros y ordena la informacion por el numero de la encuesta
vl_df_filtro_indicador_1 = (vl_df_insumo_principal[vl_df_insumo_principal["PreguntaDescNivel3"].str.lower().str.startswith('pleges')])
vl_df_filtro_indicador_1.sort_values("SbjNum", ascending=True)
vl_df_insumo_principal = vl_df_filtro_indicador_1

print(f"\n{vl_df_filtro_indicador_1.head(3)}")
print(f"\nINSUMO DATA CRUDA\n{vl_df_insumo_principal.dtypes}")


    OlaID_Padre         Ola_Padre     SbjNum      FechaEvaluacion  PaisID  \
31       202002  Ola Febrero 2020  124638767  2020-02-03 12:02:11       2   
34       202002  Ola Febrero 2020  125388797  2020-02-09 20:23:36       2   
45       202002  Ola Febrero 2020  126993584  2020-02-24 15:15:56       2   

          Pais  DepartamentoId     Departamento  CiudadID           Ciudad  \
31  Costa Rica               0  Sin Identificar         0  Sin Identificar   
34  Costa Rica               4       Guanacaste         4       Guanacaste   
45  Costa Rica               5         Heredia          5          Heredia   

    ...  PreguntaIDNivel6 PreguntaDescNivel6 PreguntaIDNivel10  \
31  ...                 0                 L2                 0   
34  ...                 0                SM1                 0   
45  ...                 0                MA1                 0   

   PreguntaDescNivel10  RespuestaID    FechaOla Flag Duracion  \
31           DESCUENTO            5  2020-02-01

In [ ]:
#Escritura de archivos en la zona silver
print(fu_escribirarchivozonasilver(vl_df_insumo_principal, "insumo_principal", 2))
#print(fu_escribirarchivozonasilver(vl_df_maestro_pdv, "insumo_pdv", 2))
#print(fu_escribirarchivozonasilver(vl_df_maestro_ubicaciones, "maestro_ubicaciones", 2))
#print(fu_escribirarchivozonasilver(vl_df_maestro_procesamientoxpais, "maestro_procesamiento_por_pais", 2))
#print(fu_escribirarchivozonasilver(vl_df_maestro_preguntasporprocesamiento, "maestro_preguntas_por_procesamiento", 2))
#print(fu_escribirarchivozonasilver(vl_df_maestro_agrupamientoporprocesamiento, "maestro_agrupamiento_por_procesamiento", 3))

In [ ]:
vl_df_insumo_principal, vl_str_mensaje = fu_leerarchivozonasilver("insumo_principal", 2)
print(vl_str_mensaje)

In [169]:
#Validación y transformación de insumos y maestros
#Datos detallados
vl_str_sentenciasql = 'SELECT OlaID_Padre id_mes_procesamiento, FechaOla fecha_procesamiento, Ola_Padre descripcion_fecha_procesamiento, ' \
                     '        pdvid id_pdv, PDV nombre_pdv, '\
                     '        PaisID id_pais, Pais descripcion_pais, Canal zona_pdv, PreguntaDescNivel1 seccion_almacen, '\
                     '        PreguntaID id_pregunta, Pregunta descripcion_pregunta, '\
                     '        PreguntaDescNivel5 promesa_compra, '\
                     '        cast(sum(case when Respuesta in ("4","5") then 1 else 0 end) as float) universo_respuestas_exitosas, '\
                     '        cast(count(Respuesta) as float) universo_respuestas, '\
                     '        cast((cast(sum(case when Respuesta in ("4","5") then 1 else 0 end) as float) / cast(count(Respuesta) as float) * 100) as float) calificacion_respuestas '\
                     ' FROM   vl_df_insumo_principal as m '\
                     ' WHERE  PreguntaDescNivel3 = "Pleges" '\
                     '        AND PreguntaDescNivel1 <> "" '\
                     '        AND (pdvid IS NOT NULL OR PDV IS NOT NULL) '\
                     ' GROUP  BY OlaID_Padre, Ola_Padre, PaisID, '\
                     '        Pregunta, Pais, FechaOla, '\
                     '        PreguntaDescNivel1, PreguntaDescNivel5 '\
                     ' HAVING cast((cast(sum(case when Respuesta in ("4","5") then 1 else 0 end) as float) / cast(count(Respuesta) as float) * 100) as float) <> 0 '
vl_df_resultado_detallado = sqldf(vl_str_sentenciasql, locals())

#Datos consolidados
vl_str_sentenciasql = 'SELECT OlaID_Padre id_mes_procesamiento, FechaOla fecha_procesamiento, Ola_Padre descripcion_fecha_procesamiento, ' \
                     '        cast(0 as int) id_pdv, "Total País" nombre_pdv, '\
                     '        PaisID id_pais, Pais descripcion_pais, "Total País" zona_pdv, "Total País" seccion_almacen, '\
                     '        PreguntaID id_pregunta, Pregunta descripcion_pregunta, '\
                     '        PreguntaDescNivel5 promesa_compra, '\
                     '        cast(sum(case when Respuesta in ("4","5") then 1 else 0 end) as float) universo_respuestas_exitosas, '\
                     '        cast(count(Respuesta) as float) universo_respuestas, '\
                     '        cast((cast(sum(case when Respuesta in ("4","5") then 1 else 0 end) as float) / cast(count(Respuesta) as float) * 100) as float) calificacion_respuestas '\
                     ' FROM   vl_df_insumo_principal as m '\
                     ' WHERE  PreguntaDescNivel3 = "Pleges" '\
                     '        AND PreguntaDescNivel1 <> "" '\
                     '        AND (pdvid IS NOT NULL OR PDV IS NOT NULL) '\
                     ' GROUP  BY OlaID_Padre, Ola_Padre, PaisID, '\
                     '        Pregunta, Pais, FechaOla, '\
                     '        PreguntaDescNivel1, PreguntaDescNivel5 '\
                     ' HAVING cast((cast(sum(case when Respuesta in ("4","5") then 1 else 0 end) as float) / cast(count(Respuesta) as float) * 100) as float) <> 0 '
vl_df_resultado_total_pais = sqldf(vl_str_sentenciasql, locals())

#Se concatenan los dos Dataframe y se elimina la columna index que se genero automaticamente y se reemplaza por la columna que quiero utilizar como indice
vl_df_resultado_indicador_1 = pd.concat([vl_df_resultado_total_pais,vl_df_resultado_detallado])
vl_df_resultado_indicador_1 = vl_df_resultado_indicador_1.set_index('id_mes_procesamiento') # Se asigna el indice del dataframe a otra columna

vl_df_resultado_indicador_1.head()

,fecha_procesamiento,descripcion_fecha_procesamiento,id_pdv,nombre_pdv,id_pais,descripcion_pais,zona_pdv,seccion_almacen,id_pregunta,descripcion_pregunta,promesa_compra,universo_respuestas_exitosas,universo_respuestas,calificacion_respuestas
id_mes_procesamiento,,,,,,,,,,,,,,
202002,2020-02-01,Ola Febrero 2020,0,Total País,2,Costa Rica,Total País,Total País,259,Encontré el precio más bajo que el resto de lo...,PRECIO BAJO SIEMPRE,1.0,1.0,100.0
202002,2020-02-01,Ola Febrero 2020,0,Total País,2,Costa Rica,Total País,Total País,166,Encontré las frutas y verduras que buscaba,ENCONTRÉ LOS PRODUCTOS QUE BUSCABA,1.0,1.0,100.0
202002,2020-02-01,Ola Febrero 2020,0,Total País,2,Costa Rica,Total País,Total País,268,Encontré las marcas que me gustan y compro,JUSTO LO QUE NECESITAS,1.0,1.0,100.0
202002,2020-02-01,Ola Febrero 2020,0,Total País,2,Costa Rica,Total País,Total País,268,Encontré lo que necesitaba para mi despensa,JUSTO LO QUE NECESITAS,1.0,1.0,100.0
202002,2020-02-01,Ola Febrero 2020,0,Total País,2,Costa Rica,Total País,Total País,277,Esperé poco tiempo en cajas,AQUÍ ME SIENTO BIEN,1.0,1.0,100.0


In [170]:
#Escribe los archivos resultados en la zona gold
print(fu_escribirarchivozonagold(vl_df_resultado_indicador_1, "indicador_1", 2))
print(fu_escribirarchivozonagold(vl_df_resultado_indicador_1, "indicador_1", 4, nombrehojaexcel="Resultado"))

Archivo "Dataset/almacenamiento/gold/indicador_1.parquet" se escribio satisfactoriamente: 
(FI, FF, T) 2022-02-10 22:22:15.673386, 2022-02-10 22:22:15.686970, 0:00:00.013584
Filas/columnas: (14, 14)

Archivo "Dataset/almacenamiento/gold/indicador_1.xlsx" se escribio satisfactoriamente: 
(FI, FF, T) 2022-02-10 22:22:15.686970, 2022-02-10 22:22:15.709790, 0:00:00.022820
Filas/columnas: (14, 14)



In [ ]:
#vl_df_insumo_principal.head(3)